# Sesión 6 - Entrenar Word2vec y Doc2Vec desde 0

En este notebook vamos a ver cómo se entrena un modelo sencillo de word2vec eligiendo las dimensiones de los vectores.

Además, se creará un modelo de Doc2Vec a partir de un conjunto de documentos y se verá cómo se puede obtener las similitudes entre documentos.


In [ ]:
# Instalamos gensim si no lo tenemos instalado
!pip3 install -U gensim
# Esto es por si no está ya instalado
!pip3 install -U pandas
!pip3 install -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 665.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 69.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


# Apartado 1.1 Descargamos un corpus de prueba

Vamos a probar con un corpus de noticias que se encuentra en la URL https://valencia.inf.um.es/valencia-tgine/corpusNoticias.zip

In [ ]:
# Descargamos un corpus de noticias que he creado
!wget --no-check-certificate https://valencia.inf.um.es/valencia-plne/corpusNoticias.zip
!unzip corpusNoticias.zip > extract.log

--2025-03-17 16:06:29--  https://valencia.inf.um.es/valencia-plne/corpusNoticias.zip
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4170052 (4.0M) [application/zip]
Saving to: ‘corpusNoticias.zip’

corpusNoticias.zip  100%[===================>]   3.98M  4.14MB/s    in 1.0s    

2025-03-17 16:06:30 (4.14 MB/s) - ‘corpusNoticias.zip’ saved [4170052/4170052]



Leemos todos lo ficheros y los metemos en una variable *texts*

Tened en cuenta que la codificación de caracteres en estos ficheros es UTF-8. Esto depende del contenido de las web a descargar.

In [ ]:
from os import listdir
from os.path import isfile, join

my_path = "corpusNoticias/"
texts = []
for fn in listdir(my_path):
  f = open(my_path+fn, encoding = "utf-8")
  file_content = f.read()
  texts.append(file_content)
  f.close()

# Comprobar que se ha leído bien:
for text in texts[:3]:
  print(text)
  print("----------"*10)

Dos muertos y cuatro heridos en un tiroteo en un centro comercial de Estados Unidos
Dos personas fueron asesinadas y otras cuatro heridas en un tiroteo en un centro comercial en Boise, en Idaho, noroeste de Estados Unidos, informó este lunes la policía. La policía llegó al centro comercial atendiendo reportes de que "al menos una persona fue disparada y derribada", informó el jefe de policía Ryan Lee a los periodistas. Los oficiales intercambiaron disparos con un hombre que tenía la misma descripción que recibieron en la llamada de emergencia. Uno de los policías fue herido. El sospechoso fue llevado bajo custodia y "no hay más amenazas a la comunidad por este incidente", confirmó Lee. "No podemos en este momento hablar sobre las motivaciones detrás del incidente, o nada más que pueda impactar la investigación", añadió. El diario The New York Times informó que el centro comercial de dos pisos Boise Towne Square, en Boise, tiene más de 150 tiendas y restaurantes. No fueron reveladas las

# Apartado 1.2 Entrenamos un modelo word2vec a partir del corpus

Aquí vamos a entrenar un modelo word2vec con la librería GENSIM. Como Tokenizer se utilizará el word_tokenize de NLTK, pero se podría usar cualquier otro tokenizador.

In [ ]:
!pip install nltk

In [ ]:
from os import listdir
from os.path import isfile, join
import numpy
import pandas
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors
from gensim.test.utils import datapath
import re
import unicodedata
from tqdm import tqdm
import gensim
import multiprocessing
import random
from nltk.tokenize import word_tokenize


In [ ]:
# Procesamos todos los textos y le aplicamos el word_tokenize de NLTK
nltk.download('punkt_tab')
train_texts=[]
for text in texts:
     train_texts.append(word_tokenize(text.lower()))

# Comprobar tokenización:
print()
print()
for t in train_texts[:3]:
  print(t)
  print("----------"*10)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['dos', 'muertos', 'y', 'cuatro', 'heridos', 'en', 'un', 'tiroteo', 'en', 'un', 'centro', 'comercial', 'de', 'estados', 'unidos', 'dos', 'personas', 'fueron', 'asesinadas', 'y', 'otras', 'cuatro', 'heridas', 'en', 'un', 'tiroteo', 'en', 'un', 'centro', 'comercial', 'en', 'boise', ',', 'en', 'idaho', ',', 'noroeste', 'de', 'estados', 'unidos', ',', 'informó', 'este', 'lunes', 'la', 'policía', '.', 'la', 'policía', 'llegó', 'al', 'centro', 'comercial', 'atendiendo', 'reportes', 'de', 'que', '``', 'al', 'menos', 'una', 'persona', 'fue', 'disparada', 'y', 'derribada', "''", ',', 'informó', 'el', 'jefe', 'de', 'policía', 'ryan', 'lee', 'a', 'los', 'periodistas', '.', 'los', 'oficiales', 'intercambiaron', 'disparos', 'con', 'un', 'hombre', 'que', 'tenía', 'la', 'misma', 'descripción', 'que', 'recibieron', 'en', 'la', 'llamada', 'de', 'emergencia', '.', 'uno', 'de', 'los', 'policías', 'fue', 'herido', '.', 'el', 'sospechoso', 'fue', 'llevado', 'bajo', 'custodia', 'y', '``', 'no', 'hay', 'más'

In [ ]:
from gensim.models import Word2Vec
# define training data
# train model
# se puede entrenar el modelo con distintos parámetros como el tamaño del vector,
# tamaño de la ventana, las veces que debe aparecer una palabra, etc.
model = Word2Vec(train_texts, vector_size=100, window=10, min_count=1, workers=10)
# summarize the loaded model
print(model)
# save model
model.save('model.bin')


Word2Vec<vocab=65286, vector_size=100, alpha=0.025>


In [ ]:
# Cargamos el modelo guardado
new_model = Word2Vec.load('model.bin')

# Probamos el nuevo modelo
# Imprimimos el vector de la palabra 'energía'
print(model.wv['energía'])


[-0.84359187  0.22467358  0.05202297  0.16554634  0.43448073 -0.51898843
 -0.15264997  1.5213182  -0.07983488 -0.8570394   0.05498248 -1.0612081
  0.37139836  0.13328692  0.2771717  -0.14330024  1.353897   -0.34903678
 -0.95016634 -0.58894944  0.24373196  1.0335588   0.84536266 -0.67604053
  0.39286238 -1.0820606  -0.7598596   0.6952372   0.20399721  0.43293494
  0.7380067  -0.2741573  -0.15049306 -0.02619279 -0.43092498  1.1428088
  0.4369473   0.5216575  -0.10490415 -0.06425738  1.0408417   0.04995571
 -0.6151536  -0.54208463  0.9406884   0.09075529  0.19669437 -0.09362527
  0.526893   -0.17861886  0.67543447  0.24991845 -0.13019685 -0.5212753
  0.08686591  1.043626    0.00707174  0.23576772  1.0676789   0.34982026
  0.13072883 -0.84579194  0.79335266  0.75567263 -0.6501041   0.94530547
  0.1115368   0.32280487 -1.0635351   0.8325982  -0.06648438 -0.6152903
  0.7327884  -0.08579908  0.38567266 -0.41000018  0.58832765 -0.1263185
 -0.33180568  0.8658027  -1.2048192  -0.34167227  0.4968

In [ ]:
# Probamos similitudes
new_model.wv.similarity("coronavirus", "covid")

0.95951295

In [ ]:
# Probamos en listar alguna de las palabras más similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'energía'
palabra = 'energía'
print(new_model.wv.most_similar(palabra))

[('coronavirus', 0.9595129489898682), ('covid-19', 0.9551697969436646), ('contagio', 0.9329785108566284), ('vacunación', 0.8813939094543457), ('sars-cov-2', 0.8801690936088562), ('inmunización', 0.8752425312995911), ('riesgo', 0.8624672293663025), ('pfizer', 0.8618341684341431), ('datos', 0.8587706685066223), ('dosis', 0.8519707322120667)]
[('reducir', 0.914434015750885), ('descarbonización', 0.909905195236206), ('cargadores', 0.9051271080970764), ('eficiencia', 0.9014433026313782), ('producción', 0.8984193801879883), ('eléctrica', 0.8964977264404297), ('inversión', 0.8935443162918091), ('emisiones', 0.8890010118484497), ('infraestructuras', 0.8875113129615784), ('transporte', 0.8832638263702393)]


In [ ]:
# Probamos alguna analogía
# Covid es a Vacunas lo que Salud es a ...
print(new_model.wv.most_similar(positive=["salud", "vacunas"], negative=["covid"], topn=10))

# Covid es a Vacunas lo que Guerra es a ...
print(new_model.wv.most_similar(positive=["guerra", "vacunas"], negative=["covid"], topn=10))


[('pública', 0.8278481364250183), ('administración', 0.7910496592521667), ('seguridad', 0.7856331467628479), ('medidas', 0.7851464748382568), ('sanidad', 0.7833719253540039), ('variantes', 0.7761925458908081), ('prevención', 0.7753053307533264), ('cavaleri', 0.7696025967597961), ('restrictivas', 0.7582018971443176), ('oms', 0.7528419494628906)]
[('importancia', 0.8727989196777344), ('agilizar', 0.8708047270774841), ('monetaria', 0.8614347577095032), ('institución', 0.8482294082641602), ('convivencia', 0.8471152186393738), ('estrategia', 0.8470028638839722), ('económica', 0.8469110727310181), ('invasión', 0.8463584184646606), ('normativa', 0.8393855690956116), ('sincera', 0.8371443152427673)]


In [ ]:
# Probamos a mostrar términos similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'ucrania'
palabra = 'ucrania'
print(new_model.wv.most_similar(palabra))

[('coronavirus', 0.9595129489898682), ('covid-19', 0.9551697969436646), ('contagio', 0.9329785108566284), ('vacunación', 0.8813939094543457), ('sars-cov-2', 0.8801690936088562), ('inmunización', 0.8752425312995911), ('riesgo', 0.8624672293663025), ('pfizer', 0.8618341684341431), ('datos', 0.8587706685066223), ('dosis', 0.8519707322120667)]
[('rusia', 0.8908936977386475), ('bruselas', 0.8861719369888306), ('cop26', 0.8851795196533203), ('insuficiente', 0.878365695476532), ('puntualizó', 0.8752503991127014), ('urgente', 0.8730335831642151), ('investigar', 0.8723407983779907), ('mitigar', 0.8718651533126831), ('violencia', 0.8716721534729004), ('afganistán', 0.8707651495933533)]


# Apartado 1.3 Entrenamos un Doc2Vec con los mismos textos

El Doc2Vec se puede ver como un tipo de sentence embeddings que traduce todo el texto a un vector de unas dimensiones determinadas.


In [ ]:
#Import all the dependencies
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

#Necestiamos crear un TaggedDocument para cada uno de los textos indicando un índice de cada texto
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(train_texts)]

# Comprobar tokenización:
print()
print()
for td in tagged_data[:3]:
  print(td)
  print("----------"*10)



TaggedDocument<['dos', 'muertos', 'y', 'cuatro', 'heridos', 'en', 'un', 'tiroteo', 'en', 'un', 'centro', 'comercial', 'de', 'estados', 'unidos', 'dos', 'personas', 'fueron', 'asesinadas', 'y', 'otras', 'cuatro', 'heridas', 'en', 'un', 'tiroteo', 'en', 'un', 'centro', 'comercial', 'en', 'boise', ',', 'en', 'idaho', ',', 'noroeste', 'de', 'estados', 'unidos', ',', 'informó', 'este', 'lunes', 'la', 'policía', '.', 'la', 'policía', 'llegó', 'al', 'centro', 'comercial', 'atendiendo', 'reportes', 'de', 'que', '``', 'al', 'menos', 'una', 'persona', 'fue', 'disparada', 'y', 'derribada', "''", ',', 'informó', 'el', 'jefe', 'de', 'policía', 'ryan', 'lee', 'a', 'los', 'periodistas', '.', 'los', 'oficiales', 'intercambiaron', 'disparos', 'con', 'un', 'hombre', 'que', 'tenía', 'la', 'misma', 'descripción', 'que', 'recibieron', 'en', 'la', 'llamada', 'de', 'emergencia', '.', 'uno', 'de', 'los', 'policías', 'fue', 'herido', '.', 'el', 'sospechoso', 'fue', 'llevado', 'bajo', 'custodia', 'y', '``', '

In [ ]:
# Definimos los parámetros de entrenamiento y entrenamos
max_epochs = 5
vec_size = 100
alpha = 0.025

doc2vec_model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm = 1,
                epochs = max_epochs)

doc2vec_model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    doc2vec_model.train(tagged_data,
                total_examples=doc2vec_model.corpus_count,
                epochs=doc2vec_model.epochs)
    # decrease the learning rate
    doc2vec_model.alpha -= 0.0002
    # fix the learning rate, no decay
    doc2vec_model.min_alpha = model.alpha

doc2vec_model.save("d2v.model")
print("Model Saved")

Model Saved


In [ ]:
from gensim.models import Doc2Vec

doc2vec_model= Doc2Vec.load("d2v.model")
# Probamos a encontrar textos similares a uno dado
query_text = "nadal"
test_data = word_tokenize(query_text)
v1 = doc2vec_model.infer_vector(test_data)

# Encontramos los documentos más similares
similar_doc = doc2vec_model.dv.most_similar(v1)

# Imprimimos los 5 documentos más similares
top5_similar_doc = similar_doc[:5]
print(top5_similar_doc)
for doc in top5_similar_doc:
  print("--------------------------")
  print(texts[int(doc[0])])
  print('Similitud:',doc[1])



[('543', 0.6218556761741638), ('172', 0.6149080395698547), ('183', 0.6147438883781433), ('1154', 0.6119747757911682), ('555', 0.6014217734336853)]
--------------------------
Djokovic vuelve en Tel Aviv a la competición dos meses después
El pasado 10 de julio, poco después de ganar su séptimo Wimbledon y vigesimoprimer Grand Slam, Novak Djokovic ya dejaba claro que no iba a dar un solo paso atrás en su beligerante actitud contra la vacunación frente al covid y que tenía escasas esperanzas de poder disputar el Abierto de Estados Unidos. Dicho y hecho. Al balcánico no le ha importado renunciar a la posibilidad de cazar a Nadal llevándose el título en Nueva York ni seguir desplomándose en el ránking tras no defender la final de 2021 en Flushing Meadows. De entonces acá, Djokovic sólo se ha dejado ver en la despedida de Roger Federer en la Laver Cup, torneo de exhibición por equipos donde ganó a Frances Tiafoe y perdió ante Felix Auger-Aliassime, quejándose de problemas en la muñeca derecha

Podemos ahora sacar el vector de una frase de ejemplo con el método ```infer_vector``` del modelo Doc2Vec

In [ ]:
frase = "Las vacunas son una muy buena solución para el COVID"
vector = doc2vec_model.infer_vector(word_tokenize(frase))
print(vector)



[ 0.01148563 -0.13896534 -0.03604966 -0.02097633  0.00653889 -0.05962177
  0.06393418  0.04876735 -0.19678718 -0.07372516 -0.01040513  0.0003279
 -0.09576906  0.05710362  0.12689795 -0.02119126  0.07966362 -0.17777073
 -0.05198049 -0.17349407  0.1410726   0.05685994  0.04120975  0.02399118
 -0.0089343   0.07324933 -0.14778252 -0.1044274  -0.15853967 -0.11595079
  0.07016157  0.04665369  0.14071223 -0.06276125 -0.13366948  0.09746307
 -0.0587103  -0.03342016 -0.06071347 -0.00668062  0.09126469  0.05591986
 -0.08436327 -0.06714002  0.01950441  0.08597095 -0.14258492  0.00782298
  0.17515711 -0.03493492 -0.05566042 -0.08215437  0.01072543 -0.09646109
 -0.13831706 -0.08581062  0.04258046  0.03053954 -0.0585182   0.1317613
  0.07788628  0.03111159 -0.03090247  0.02157467 -0.02006445  0.07024138
  0.0455654   0.14937687 -0.12429123  0.10729324  0.05234659  0.03596282
  0.10249898 -0.13140701  0.07290321 -0.07057644  0.13738428 -0.0141278
  0.00126637 -0.14657336 -0.02480177 -0.00623553 -0.17

##Ejercicio a resolver

### 1.- Con el nuevo Doc2Vec entrenado para generar sentence embeddings probar cómo funciona la clasificación de los sentence embeddings del conjunto de entrenamiento `dataset_train.csv`y de `dataset_test`.

(Si no sabe cómo empezar, veáse apartado 1.8 de práctica 6_1_Word_embeddings ...)

In [ ]:
# Descargamos los datasets en español que hemos usado en otras prácticas
!wget -c --no-check-certificate https://valencia.inf.um.es/valencia-plne/dataset_train.csv
!wget -c --no-check-certificate https://valencia.inf.um.es/valencia-plne/dataset_test.csv


--2025-03-17 16:14:27--  https://valencia.inf.um.es/valencia-plne/dataset_train.csv
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1367959 (1.3M) [text/csv]
Saving to: ‘dataset_train.csv’

dataset_train.csv   100%[===================>]   1.30M  1.63MB/s    in 0.8s    

2025-03-17 16:14:28 (1.63 MB/s) - ‘dataset_train.csv’ saved [1367959/1367959]

--2025-03-17 16:14:28--  https://valencia.inf.um.es/valencia-plne/dataset_test.csv
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 584195 (571K) [text/csv]
Saving to: ‘dataset_test.csv’

dataset_test.csv    100%[===================>] 570.50K   993KB/s    in 0.6s    

2025-03-17 16:14:29 (993 KB/s) - ‘dataset_test.csv’ save

In [ ]:
import pandas
import numpy as np

df_train = pandas.read_csv("dataset_train.csv",encoding="UTF-8")
df_test = pandas.read_csv("dataset_test.csv",encoding="UTF-8")

# Ponemos en lower_case los dos conjuntos de tweets
# ...

Ejemplos usados para entrenar:  4171
Ejemplos usados para test:  1788
Resultados ----- Accuracy: 0.7483221476510067
              precision    recall  f1-score   support

    negative       0.67      0.39      0.49       561
    positive       0.77      0.91      0.83      1227

    accuracy                           0.75      1788
   macro avg       0.72      0.65      0.66      1788
weighted avg       0.74      0.75      0.73      1788



### 2.- Entrena ahora otro Doc2Vec ampliando las dimensiones y compara los resultados.

In [ ]:
#Entrenamos el nuevo Doc2Vec con 300 dimensiones

# Definimos los parámetros de entrenamiento y entrenamos
max_epochs = 10
vec_size = 200
alpha = 0.025



In [ ]:
from sklearn.svm import LinearSVC


Resultados ----- Accuracy: 0.7572706935123042
              precision    recall  f1-score   support

    negative       0.66      0.47      0.55       561
    positive       0.79      0.89      0.83      1227

    accuracy                           0.76      1788
   macro avg       0.72      0.68      0.69      1788
weighted avg       0.75      0.76      0.74      1788

